<a href="https://colab.research.google.com/github/kyle-gao/ML_ipynb/blob/master/PyTorchHuggingFace_RoBERTa_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#I ran into alot of GPU memory issues with this one.

In [ ]:
!pip install datasets
!pip install transformers #from huggingface

In [1]:
import torch
from transformers import RobertaForSequenceClassification, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import time

In [2]:
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 3

In [3]:
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [5]:
(train_dataset,valid_dataset) = load_dataset('glue', 'mrpc', split=('train','validation')) #dictionary of lists
train_dataset = train_dataset.map(lambda data: tokenizer(data['sentence1'], data['sentence2'], padding='max_length',truncation=True), batched=True) 
val_dataset = valid_dataset.map(lambda data: tokenizer(data['sentence1'], data['sentence2'], padding='max_length',truncation=True), batched=True) 
train_dataset.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])
#pads to 512

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-9ea168b9bdc8f03a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-45e6d783c772c766.arrow


In [6]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True) #dict with keys 'attention_mask','input_ids','label'
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=True)

In [7]:
device = torch.device('cuda');
roberta.to(device)
roberta.train();

Pytorch training loop

In [8]:
optim = AdamW(roberta.parameters(), lr=1e-5)
for epoch in range(EPOCHS):
    batch_num = 0
    start_time = time.time()
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = roberta(input_ids, attention_mask=attention_mask, labels=labels) #tuple (loss,output)
        loss = outputs[0]
        predictions = outputs[1].argmax(axis=1)
        acc = sum(labels == predictions).item()/(TRAIN_BATCH_SIZE)
        loss.backward()
        optim.step()
        batch_num +=1
        if (batch_num) %25 == 0:
          print(f"epoch: {epoch}, batch number: {batch_num}, loss={loss}, acc={acc}") #batch acc, probably should write use running accuracy
    print(f"epoch time:{time.time()-start_time}")

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


epoch: 0, batch number: 25, loss=0.562017560005188, acc=0.6875
epoch: 0, batch number: 50, loss=0.5443019866943359, acc=0.75
epoch: 0, batch number: 75, loss=0.6674498319625854, acc=0.5625
epoch: 0, batch number: 100, loss=0.7832850813865662, acc=0.4375
epoch: 0, batch number: 125, loss=0.6366065144538879, acc=0.5
epoch: 0, batch number: 150, loss=0.34386757016181946, acc=0.75
epoch: 0, batch number: 175, loss=0.30218690633773804, acc=0.875
epoch: 0, batch number: 200, loss=0.4022648334503174, acc=0.8125
epoch: 0, batch number: 225, loss=0.5034917593002319, acc=0.75
epoch time:198.80919814109802
epoch: 1, batch number: 25, loss=0.36211881041526794, acc=0.8125
epoch: 1, batch number: 50, loss=0.39435672760009766, acc=0.875
epoch: 1, batch number: 75, loss=0.35503843426704407, acc=0.8125
epoch: 1, batch number: 100, loss=0.36488252878189087, acc=0.8125
epoch: 1, batch number: 125, loss=0.37262025475502014, acc=0.875
epoch: 1, batch number: 150, loss=0.14499156177043915, acc=1.0
epoch: 1,

In [12]:
torch.save(roberta.state_dict(), "/Roberta_GLUE_Mrpc")

Evaluation

In [20]:
device = torch.device('cpu');
roberta.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
acc=0
batch_num=0
for batch in val_loader:
      if torch.cuda.is_available():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

      outputs = roberta(input_ids, attention_mask=attention_mask, labels=labels) #tuple (loss,output)
      predictions = outputs[1].argmax(axis=1)
      acc += sum(labels == predictions).item()/(TRAIN_BATCH_SIZE)
      batch_num +=1
acc/batch_num